In [ ]:
#https://ericmjl.github.io/score-models/notebooks/01-score-function.html#fig-likelihood

In [ ]:
from jax import random

key = random.PRNGKey(44)

true_mu = 3.0
true_sigma = 1.0

data = random.normal(key, shape=(1000, 1)) * true_sigma + true_mu
data[0:5]  # showing just the first 10 samples drawn

In [100]:
from score_models.models.gaussian import GaussianModel
from inspect import getsource

print(getsource(GaussianModel))

class GaussianModel(eqx.Module):
    """Univariate Gaussian score function."""

    mu: np.array = np.array(0.0)
    log_sigma: np.array = np.array(0.0)

    @eqx.filter_jit
    def __call__(self, x):
        """Forward pass.

        :param x: Data. Should be of shape (1, :),
            where `1` is in the batch dimension slot.
            as the model is intended to be vmapped over batches of data.
        :returns: Score of a Gaussian conditioned on a `mu` and `log_sigma`.
        """
        gaussian_score_func = jacfwd(norm.logpdf)
        return gaussian_score_func(x, loc=self.mu, scale=np.exp(self.log_sigma))



In [101]:
gaussian_model = GaussianModel()
(
    gaussian_model(-3.0),
    gaussian_model(0.0),
    gaussian_model(3.0),
)

(Array(3., dtype=float32, weak_type=True),
 Array(-0., dtype=float32, weak_type=True),
 Array(-3., dtype=float32, weak_type=True))

In [103]:
from jax import vmap
data_score = vmap(gaussian_model)(data).squeeze()
data_score[0:5]

Array([-0.8219464, -3.8602278, -1.4089172, -3.4423368, -3.2420166],      dtype=float32)

In [116]:
true_sigma

1.0

In [ ]:
from matplotlib import pyplot as plt
support = jax.numpy.linspace(0,6,100)
plt.plot(support, vmap(GaussianModel(true_mu,jax.numpy.log(true_sigma)))(support).squeeze(),label='true')
plt.plot(support, vmap(GaussianModel())(support).squeeze(),label='init')
plt.legend()